In [1]:
import os
import shutil
import weave
import time

import matplotlib as plt

from dummy_baskets import generate_dummy_baskets, run_index_basket_upload_test
from weave.index.index_pandas import IndexPandas
from weave.index.index_sqlite import IndexSQLite
from weave.index.index_sql import IndexSQL
from fsspec.implementations.local import LocalFileSystem
from dask import delayed, compute
from dask.distributed import Client, LocalCluster
import dask
import dask.array as da
import dask.bag as db

In [19]:
# Set up a local cluster and client with 5 concurrent processes
cluster = LocalCluster(n_workers=1, threads_per_worker=1, processes=True)
client = Client(cluster)

c:\Users\carso\miniconda3\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53561 instead
  warnings.warn(


In [ ]:
baskets = generate_dummy_baskets(basket_count=1000, file_count=1, file_size_mb=1, file_path="dummy_txt", num_basket_types=100)

# Create a delayed function to upload all of our chunks
@delayed
def upload_chunk(i, chunk):
    pantry = weave.Pantry(IndexSQLite, pantry_path=f"dummy_pantry_{i}", file_system=LocalFileSystem())
    run_index_basket_upload_test(chunk, pantry)

# Create 5 equal chunks each with 200 baskets
chunks = [baskets[i:i+200] for i in range(0, len(baskets), 200)]

# Upload all chunks in parallel
delayed_uploads = [upload_chunk(i, chunk) for i, chunk in enumerate(chunks)]

start = time.time()
results = client.compute(delayed_uploads, sync=True)
end = time.time()
print(end - start)


8.727205514907837


In [20]:
baskets = generate_dummy_baskets(basket_count=1000, file_count=1, file_size_mb=1, file_path="dummy_txt", num_basket_types=1)
pantry = weave.Pantry(IndexPandas, pantry_path="dummy_pantry", file_system=LocalFileSystem())

# Create a delayed function to upload all of our chunks
@delayed
def upload_chunk(chunk):
    pantry = weave.Pantry(IndexPandas, pantry_path="dummy_pantry", file_system=LocalFileSystem())
    run_index_basket_upload_test(chunk, pantry)

# Create 5 equal chunks each with 200 baskets
chunks = [baskets[i:i+200] for i in range(0, len(baskets), 200)]

# Upload all chunks in parallel
delayed_uploads = [upload_chunk(chunk) for chunk in chunks]

start = time.time()
results = client.compute(delayed_uploads, sync=True)
end = time.time()
print(end - start)

KeyboardInterrupt: 

In [18]:
shutil.rmtree("dummy_txt")
shutil.rmtree("dummy_pantry")
pantry.index.drop_index()

AttributeError: 'IndexPandas' object has no attribute 'drop_index'

In [9]:
# Tear down all of the pantries and indexes, along with the dummy file directory
shutil.rmtree("dummy_txt")

for i in range(5):
    pantry = weave.Pantry(IndexSQLite, pantry_path=f"dummy_pantry_{i}", file_system=LocalFileSystem())
    if str(pantry.index) == "IndexSQLite" or str(pantry.index) == "IndexSQL":
        pantry.index.drop_index()
    shutil.rmtree(f"dummy_pantry_{i}")

In [5]:
# Shutdown the client
client.shutdown()

In [4]:
baskets = generate_dummy_baskets(basket_count=1000, file_count=1, file_size_mb=1, file_path="dummy_txt", num_basket_types=5)
pantry = weave.Pantry(IndexSQLite, pantry_path="dummy_pantry", file_system=LocalFileSystem())
print(pantry.index)

cluster = LocalCluster(n_workers=5, threads_per_worker=1, processes=True)
client = Client(cluster)

def upload_partition(partition):
    pantry = weave.Pantry(IndexSQLite, pantry_path="dummy_pantry", file_system=LocalFileSystem())
    run_index_basket_upload_test(partition, pantry)
    return f"Uploaded {len(partition)} baskets"

start = time.time()
dask_baskets = db.from_sequence(baskets, partition_size=200)
dask_baskets.map_partitions(upload_partition).compute()
end = time.time()

print(end - start)

IndexSQLite


c:\Users\carso\miniconda3\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54912 instead
  warnings.warn(


ValueError: Unable to upload a basket to a read-only file system.